In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf

In [27]:
from pyspark.sql.types import IntegerType,StructField,ArrayType,StringType,StructType
from pyspark.sql import functions as F

In [3]:
spark=SparkSession.builder.appName('test').master('local[*]').config('spark.sql.shuffle.partitions',2).getOrCreate()

22/05/13 17:55:31 WARN Utils: Your hostname, MaQiang resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
22/05/13 17:55:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/13 17:55:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sc=spark.sparkContext

In [5]:
rdd=sc.parallelize(([1,2,3,4,5,6,7,8,9])).map(lambda x:[x])

In [6]:
df=rdd.toDF(['num'])

#  SparkSession.udf.register()  定义UDF函数

## 定义数字*10

In [7]:
def fun(x):
    return x*10

In [8]:
udf2=spark.udf.register('udf1',fun,IntegerType())

In [9]:
df.selectExpr('udf1(num)').show()

+---------+
|udf1(num)|
+---------+
|       10|
|       20|
|       30|
|       40|
|       50|
|       60|
|       70|
|       80|
|       90|
+---------+



In [10]:
df.select(udf2(df['num'])).show()

+---------+
|udf1(num)|
+---------+
|       10|
|       20|
|       30|
|       40|
|       50|
|       60|
|       70|
|       80|
|       90|
+---------+



In [11]:
udf3=F.udf(fun,IntegerType())

In [12]:
df.select(udf3(df['num'])).show()

+--------+
|fun(num)|
+--------+
|      10|
|      20|
|      30|
|      40|
|      50|
|      60|
|      70|
|      80|
|      90|
+--------+



## 定义split

In [13]:
rdd=sc.parallelize([['hadoop spark flink'],['hadoop flink java']])
df=rdd.toDF(['line'])

In [14]:
def fun1(x):
    return x.split(' ')

In [32]:
udf4=spark.udf.register('udf4',fun1,ArrayType(StringType()))

22/05/13 18:21:53 WARN SimpleFunctionRegistry: The function udf4 replaced a previously registered function.


In [18]:
df.select(udf4(df['line'])).show()

+--------------------+
|          udf4(line)|
+--------------------+
|[hadoop, spark, f...|
|[hadoop, flink, j...|
+--------------------+



In [20]:
df.createOrReplaceTempView('lines')
spark.sql('select udf4(line) from lines').show(truncate=False)

+----------------------+
|udf4(line)            |
+----------------------+
|[hadoop, spark, flink]|
|[hadoop, flink, java] |
+----------------------+



In [21]:
udf5=F.udf(fun1,ArrayType(StringType()))

df.select(udf5(df['line'])).show(truncate=False)

+----------------------+
|fun1(line)            |
+----------------------+
|[hadoop, spark, flink]|
|[hadoop, flink, java] |
+----------------------+



## udf定义字典类型

In [25]:
import string

rdd=sc.parallelize([1,2,3,4,5,6]).map(lambda x:[x])
df=rdd.toDF(['num'])

In [40]:
def fun3(x):
    return {'num':x,'letter':string.ascii_letters(x)}

In [41]:
udf6=spark.udf.register('udf6',fun3,StructType().add('num',IntegerType()).add('letter',StringType()))

22/05/13 18:26:00 WARN SimpleFunctionRegistry: The function udf6 replaced a previously registered function.


In [42]:
df.select(udf6(df['num'])).show()

22/05/13 18:26:01 ERROR Executor: Exception in task 2.0 in stage 37.0 (TID 95)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 200, in _batched
    for it

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 200, in _batched
    for item in iterator:
  File

In [43]:
df.selectExpr('udf6(num)').show()

22/05/13 18:26:02 ERROR Executor: Exception in task 1.0 in stage 39.0 (TID 99)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 200, in _batched
    for it

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/home/maqiang/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 200, in _batched
    for item in iterator:
  File